In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Input, Dropout, GlobalAveragePooling2D, AveragePooling2D, Conv2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau

In [3]:
# Reading from csv file
df = pd.read_csv('data2013.csv')
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [4]:

image_size=(48,48)
pixels = df['pixels'].tolist() # Converting the column element into a list for each row
width, height = 48, 48
faces = []

for pixel_sequence in pixels:
  face = [int(pixel) for pixel in pixel_sequence.split(' ')] # Splitting the string by space character as a list
  face = np.asarray(face).reshape(width, height) #converting the list to numpy array in size of 48*48
  face = cv2.resize(face.astype('uint8'),image_size) #resize the image to have 48 cols (width) and 48 rows (height)
  faces.append(face.astype('float32')) #makes the list of each images of 48*48 and their pixels in numpyarray form
  
faces = np.asarray(faces) #converting the list into numpy array
faces = np.expand_dims(faces, -1) #Expand the shape of an array -1=last dimension => means color space
emotions = pd.get_dummies(df['emotion']).to_numpy() #converting categorical variable type i.e., emotions into dummy/indicator variables


In [11]:
x = faces.astype('float32')
x = x / 255.0 #Dividing the pixels by 255 for normalization  => range(0,1)

# Scaling the pixels value in range(-1,1)
x = x - 0.5
x = x * 2.0

In [ ]:
num_samples, num_classes = emotions.shape

num_samples = len(x)
num_train_samples = int((1 - 0.2)*num_samples)

# Traning data
train_x = x[:num_train_samples]
train_y = emotions[:num_train_samples]

# Validation data
val_x = x[num_train_samples:]
val_y = emotions[num_train_samples:]

train_data = (train_x, train_y)
val_data = (val_x, val_y)

In [ ]:
##designing the cnn model
model = Sequential()

# 1st CNN layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(train_x.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(16, kernel_size=(7, 7), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

# 2nd CNN layer
model.add(Conv2D(32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

# 3rd CNN layer
model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

# 4th CNN layer
model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

# 5th CNN layer
model.add(Conv2D(256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=num_classes, kernel_size=(3, 3), padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax',name='predictions'))


In [ ]:
# data generator Generates batches of tensor image data with real-time data augmentation
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [ ]:

# model parameters/compilation

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 16)        50192     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 46, 46, 16)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 23, 23, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 16)        0

In [ ]:


batch_size = 128 
num_epochs = 500
verbose = 1 
num_classes = 7 
patience = 64
datasets = ['data2013']
base_path="/content"
for dataset_name in datasets:
    print('Training dataset:', dataset_name)

    #callbacks
    log_file_path = dataset_name + '_emotion_training.log'

    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,patience=int(patience/4), verbose=1)
    
    trained_models_path = base_path + dataset_name + 'simple_cnn'
    model_names = trained_models_path + '.{epoch:02d}-{val_loss:.2f}.hdf5'     
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
    my_callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

    # loading dataset
    train_faces, train_emotions = train_data
    history=model.fit(data_generator.flow(train_faces, train_emotions,
                                            batch_size),
                        epochs=num_epochs, verbose=1
                        ,callbacks=my_callbacks,validation_data =val_data)   #not callbacks = [my_callbacks] since we my_callbacks is already a list

Training dataset: data2013
Epoch 1/500
225/225 [==============================] - 67s 84ms/step - loss: 1.8462 - accuracy: 0.2483 - val_loss: 7.3687 - val_accuracy: 0.1158

Epoch 00001: val_loss improved from inf to 7.36866, saving model to /contentdata2013simple_cnn.01-7.37.hdf5
Epoch 2/500
225/225 [==============================] - 16s 71ms/step - loss: 1.6181 - accuracy: 0.3620 - val_loss: 6.8863 - val_accuracy: 0.1158

Epoch 00002: val_loss improved from 7.36866 to 6.88631, saving model to /contentdata2013simple_cnn.02-6.89.hdf5
Epoch 3/500
225/225 [==============================] - 15s 66ms/step - loss: 1.5102 - accuracy: 0.4150 - val_loss: 1.4187 - val_accuracy: 0.4539

Epoch 00003: val_loss improved from 6.88631 to 1.41872, saving model to /contentdata2013simple_cnn.03-1.42.hdf5
Epoch 4/500
225/225 [==============================] - 15s 65ms/step - loss: 1.4533 - accuracy: 0.4370 - val_loss: 1.3733 - val_accuracy: 0.4542

Epoch 00004: val_loss improved from 1.41872 to 1.37331, s

In [ ]:
#evaluate() returns [loss,acc]
score = model.evaluate(val_x, val_y, verbose=1) 
print('Test loss:', score[0])
print('Test accuracy:', score[1]*100)

225/225 [==============================] - 1s 5ms/step - loss: 0.9581 - accuracy: 0.6421
Test loss: 0.9580771327018738
Test accuracy: 64.21008706092834


In [5]:
#saving weights
model.save_weights("data.h5")

#saving architecture
model_json = model.to_json()
with open("data.json", "w") as json_file:
    json_file.write(model_json)

#model.save_weights("model.h5")
print("Saved model to disk")

NameError: name 'model' is not defined